In [ ]:
!pip install altair
!pip install altair_saver
from altair_saver import save
import altair as alt
import pandas as pd
import numpy as np

MyData = pd.read_csv("data.csv")

# Global Parameters:
Weeks = 21 # How many weeks in the dataset
Signals = list(MyData.columns)
Signals.remove('date')
Signals.remove('epiweek')
Signals.remove('region')
Signals.remove('fips')

ChartList = []

selection = alt.selection_multi(fields=['region'], bind='legend')

for signal in MyData.columns:
    if (signal in Signals):
        for subgraph in range(5):
            DataDictionary = {}
            StateList = [0]
            for state in range(10*subgraph+1,10*subgraph+11):
                StateList.append(state)
            for state in StateList: # 50 States and one total
                DataList = []
                DateList = []
                for week in range(Weeks):
                    DataList.append(MyData.loc[Weeks*state+week][signal])
                    DateList.append(MyData.loc[Weeks*state+week]["date"])
                DataDictionary[str(MyData.loc[Weeks*state+week]["region"])] = DataList

            Data = pd.DataFrame(DataDictionary,index = DateList)
            Data.columns.name = "region"
            Data.index.name = "date"
            Data = Data.reset_index().melt('date')

            chart = alt.Chart(Data).mark_line().encode(
                alt.X('date',
                    sort=DateList,
                ),
                #x='date',
                y='value',
                color='region',
                opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),           
            ).properties(
                width=300,
                height=225,
                title = signal,
            ).add_selection(
                selection
            )
            
            ChartList.append(chart)

for subgraph in range(5):
    ChartNow = ChartList[subgraph]
    for signal in range(len(Signals)):
        if (signal == 0):
            ChartNow = ChartList[5*signal+subgraph]
        else:
            ChartNow = ChartNow & ChartList[5*signal+subgraph]
    if (subgraph == 0):
        Chart = ChartNow
    else:
        Chart = Chart | ChartNow

Chart = Chart.configure_legend(
    orient='top',
    titleFontSize=20,
    labelFontSize=20,
    symbolLimit=51,
    columns=25,
).configure_title(fontSize=12)
Chart#.save("Signal.html")
